In [3]:
from selenium import webdriver # web scraping
from selenium.webdriver.support.ui import Select # web scraping
from selenium.webdriver.common.keys import Keys # web scraping
from bs4 import BeautifulSoup # tratamento de html
import pandas as pd # tratamento de dados tabulares
import re # expressoes regulares
from zipfile import ZipFile
import time

In [4]:
# PGFN

options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')

In [5]:
url1 = 'https://www.listadevedores.pgfn.gov.br/'
driver = webdriver.Chrome(options=options)
driver.get(url1)

In [29]:
# SELECT UF

UF = driver.find_element_by_id('ufInput')
options = [i.get_attribute("value") for i in UF.find_elements_by_tag_name("option")]
UF_field = Select(UF)
UF_field.select_by_value('AL') # loop in options list

In [53]:
id = ['nat_0', 'nat_1', 'nat_2', 'nat_3', 'nat_4', 'nat_5', 'nat_6']
xpaths = ['//*[@id="'+str(i)+'"]' for i in id]
nome = ['FGTS', 'PREV']

In [40]:
box = driver.find_element_by_xpath(xpaths[1])
box.is_selected()
driver.execute_script("arguments[0].click();", box)

In [41]:
# press button CONSULTAR
consult = driver.find_element_by_xpath('/html/body/dev-root/dev-consulta/main/dev-filtros/div[3]/div/div/button[1]')
consult.click()

In [42]:
export = driver.find_element_by_xpath('/html/body/dev-root/dev-consulta/main/dev-resultados/div[3]/div/button[1]')
export.click()

In [44]:
zf = ZipFile('Consulta_Lista_Devedores_2020_04_08.zip')

df_title = pd.read_fwf('Consulta_Lista_Devedores_2020_04_08.csv', sep=';', skiprows=1)

df = pd.read_csv(zf.open('Consulta_Lista_Devedores_2020_04_08.csv'), sep=';', encoding="ISO-8859-1", skiprows=9)

In [57]:
df['UF'] = df_title.iloc[0][0][-3:-1]
df = df.query('Nome.str.contains("LTDA|EIRELI")==False')
df = df[~df["CPF/CNPJ"].str.contains("\*")].reset_index(drop=1)
df['Natureza'] = nome[1]

In [59]:
df.to_csv('sample_file.csv')

In [60]:
df.head()

,CPF/CNPJ,Nome,Nome Fantasia,Valor Total,Valor da Dívida Selecionada,UF,Natureza
0,12.274.379/0001-07,LAGINHA AGRO INDUSTRIAL SA,NaN,"1.449.587.099,97","47.003.690,95",AL,PREV
1,12.275.715/0001-36,SA LEAO IMRAOS ACUCAR E ALCOOL EM RECUPERACAO...,USINA CENTRAL LEAO,"540.690.712,18","50.863.418,17",AL,PREV
2,12.607.842/0001-95,USINA SANTA CLOTILDE S A,USINA SANTA CLOTILDE,"336.649.703,21","19.022.359,70",AL,PREV
3,12.272.498/0001-20,USINA CANSANCAO DE SINIMBU SA,USINA SINIMBU,"305.468.599,83","15.002.140,55",AL,PREV
4,12.265.245/0001-20,USINAS REUNIDAS SERESTA SA EM RECUPERACAO JUD...,USINA SERESTA,"262.414.242,36","7.254.447,10",AL,PREV


In [4]:
# CVM

url2 = 'https://cvmweb.cvm.gov.br/SWB/Sistemas/SCW/CPublica/CiaAb/FormBuscaCiaAb.aspx?TipoConsult=c'

In [6]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-gpu')

In [7]:
driver1 = webdriver.Chrome(options=options)
driver1.get(url2)

In [10]:
df = pd.read_csv('sample_file.csv')

In [15]:
doc = df["CPF/CNPJ"][1]

In [17]:
doc = re.sub(r'[^\w]','', doc)

In [20]:
field = driver1.find_element_by_id('txtCNPJNome')

In [22]:
field.clear()
field.send_keys(doc)

In [23]:
cont = driver1.find_element_by_id('btnContinuar')
cont.click()

In [25]:
if driver1.find_element_by_id('lblMsg'):
    df['status'] = 'nao listada'
    print('encontrou')

encontrou


In [26]:
page = BeautifulSoup(driver1.page_source, 'lxml')

In [27]:
table = page.find('table', id='dlCiasCdCVM')

In [28]:
tabela_companhia = pd.read_html(str(table), header=0)[0]

In [29]:
tabela_companhia

,CNPJ,NOME,TIPO DE PARTICIPANTE,CÓDIGO CVM,SITUAÇÃO REGISTRO
0,34.274.233/0001-02,PETROBRAS DISTRIBUIDORA SA,CIAS ABERTAS,14249,Cancelado em 05/02/2003
1,34.274.233/0001-02,PETROBRAS DISTRIBUIDORA SA,CIAS ABERTAS,24295,Concedido em 12/12/2017
2,33.795.055/0001-94,PETROBRAS QUÍMICA S.A.- PETROQUISA,CIAS ABERTAS,13595,Cancelado em 08/11/2006
3,33.000.167/0001-01,PETRÓLEO BRASILEIRO S.A. - PETROBRAS,CIAS ABERTAS,9512,Concedido em 20/07/1977


In [30]:
tabela_companhia = tabela_companhia.query('`SITUAÇÃO REGISTRO`.str.contains("Cancelado")==False')

In [31]:
tabela_companhia

,CNPJ,NOME,TIPO DE PARTICIPANTE,CÓDIGO CVM,SITUAÇÃO REGISTRO
1,34.274.233/0001-02,PETROBRAS DISTRIBUIDORA SA,CIAS ABERTAS,24295,Concedido em 12/12/2017
3,33.000.167/0001-01,PETRÓLEO BRASILEIRO S.A. - PETROBRAS,CIAS ABERTAS,9512,Concedido em 20/07/1977


In [34]:
for i in tabela_companhia['CÓDIGO CVM'].values:
    execute action

24295

In [35]:
first_code_cvm = tabela_companhia['CÓDIGO CVM'].values[0]

In [37]:
button = driver1.find_element_by_link_text(str(first_code_cvm))

In [38]:
button.click()

In [ ]:
code1
time.sleep(5)
code2